In [504]:
%run data_plot.ipynb

In [505]:
dijkstra_data = {'best_path': {},
            'possible_paths': {}}
solutions = [{}]

In [506]:
# this is a dict that will store the best known end-to-end transmission rates for each node as we explore the space
known_costs = {}

In [507]:
# this dict contains lists containing best known paths associated with the best known costs defined above
routes = {sensor: [] for sensor in x_y_data.keys()}

In [508]:
# this list will store the nodes that we will visit
visited_nodes = []

In [509]:
# this list will store unvisited nodes
unvisited_nodes = []

In [510]:
def init_known_costs(init_node):
    """
    init_known_costs(init_node)
    initializes the starting node by setting its known cost to 0 and its best 
    path to itself. The function initializes all best known costs to nodes 
    except initial node to -infinity
    init_node: the initial node
    returns: nothing
    """
    global unvisited_nodes
    # set all known costs to nodes to -infinity and those on the initial node to 0
    for sensor in x_y_data.keys():
        if sensor != init_node:
            known_costs[sensor] = -np.inf
        else:
            known_costs[sensor] = 0
            # set known path to init node to the same init node
            routes[init_node] = [sensor]
    # add init node to visited nodes
    visited_nodes.append(init_node)
    # set unvisited nodes to all notes but the init node
    unvisited_nodes = [node for node in x_y_data.keys() if node != init_node]

    return

In [511]:
init_known_costs(9)

In [512]:
visited_nodes

[9]

In [513]:
def get_path_bandwidth_costs(current_node):
    nodes = [node for node in x_y_data.keys() if node != current_node]
    bandwidths = {}
    for node in nodes:
        distance = calculate_distance(x_y_data[current_node], x_y_data[node])
        bandwidth = convert_distance_bandwidth(distance)
        bandwidths[node] = bandwidth

    return bandwidths

In [514]:
def select_max_path(node, no_visited_nodes):
    """
    select_max_path(node)
    this is the main function that examines through nodes in a bit by bit fashion with the aim 
    of establishing the next best path. The next best path consideration is made on the basis 
    of the end-to-end transmission rate i.e. min{x1, x2, x3}. The highest value derived is 
    the best path. Throughout traversal, the variables e.g. visited_nodes, unvisited_nodes, 
    e.t.c. get updated with the latest information
    node: the current node that has been selected from the previous iteration as the best node
    returns: the best next node based on the dijkstra evaluation criteria
    """
    costs = get_path_bandwidth_costs(node)
    '''
    get the minimum value in a node and set it as the max value. 
    Since for each node, there is atleast one unreachable sensor, 
    I set the value to 0
    '''
    max_val = 0
    # get the corresponding nodes (for the bandwidth set below)
    next_nodes = []
    # Loop through each car linked to the node passed in a parameter. This information is found in the search space
    for car, bandwidth in costs.items():
        if car not in visited_nodes:
            # define path for storing bandwidths of nodes in a certain route
            path_bandwidths = []
            # loop through the nodes in currently known best route to current node and add their interconnection bandwidths
            for i in routes[node]:
                # append each computed bandwidth to the path bandwidths
                path_bandwidths.append(known_costs[i])
            # append the bandwidth to the next neighbouring node (associated to this iteration)
            path_bandwidths.append(bandwidth)
            # to exclude the source node whose cost is set to 0
            path_bandwidths = path_bandwidths[1:]
            # computed the end-to-end transmission
            end_to_end_bandwidth = min(path_bandwidths)
            if end_to_end_bandwidth == 0:
                absolute_cost = 0
            else:
                # calculate absolute cost
                absolute_cost = round(absolute_cost_function(end_to_end_bandwidth, node_to_node_latency * no_visited_nodes), 3)
            # Check if this bandwidth is the new maximum
            if absolute_cost > max_val:
                max_val = absolute_cost
                # Update next_nodes with only the current highest cost
                next_nodes = [car]
            elif absolute_cost == max_val:
                # If there is more than one node with that max value in the sub dict
                next_nodes.append(car)
            # Calculate the cummulative bandwidth to reach this neighbor
            new_cost = absolute_cost
            '''
            if the new end-to-end bandwidth is greater than the greatest known 
            bandwidth to that node, update the var and the path associated with it
            '''
            if new_cost > known_costs[car]:
                routes[car] = routes[node] + [car]
                known_costs[car] = new_cost
    # Mark the selected node as visited and remove it from unvisited nodes
    visited_nodes.append(next_nodes[0])
    no_visited_nodes = len(visited_nodes)
    unvisited_nodes.remove(next_nodes[0])
    
    # select the value in the array. If they are 2, select the first. But it honestly doesnt matter which you pick
    return next_nodes[0], no_visited_nodes

In [515]:
def get_optimal_route(destination, dijkstra_data, solutions):
    """
    get_optimal_route(destination)
    decodes the best known route the destination passed in as an 
    argument of the function
    destination: the destination node whose best path from the origin is to be represented
    returns: nothing
    """
    # instantiate a sting that will be used in parsing the best route
    optimal_route = ''
    bandwidths = []
    '''
    get the known best path associated with the node passed in as an argument.
    This loop simply parses the 'route[destination]' node to a  user friendly
    representation
    '''
    for idx, path in enumerate(routes[destination]):
        if idx > 0:
            bandwidths.append(get_path_bandwidth_costs(path)[routes[destination][idx - 1]])
        if path != destination:
            optimal_route += str(path) + " -> "
        else:
            optimal_route += str(path)
    latency = node_to_node_latency * (len(routes[destination]) - 1)
    end_to_end_bandwidth = min(bandwidths)
    absolute_cost = round(absolute_cost_function(end_to_end_bandwidth, latency), 3)
    if len(dijkstra_data['best_path']) == 0:
        solutions[0] = {"source node": f"Node-{routes[destination][0]}"}
        routing_path = ''
        for idx, sensor in enumerate(routes[destination]):
            if idx > 0:
                routing_path += f"(Node-{sensor}, {get_path_bandwidth_costs(sensor)[routes[destination][idx - 1]]} Mbps),"
        solutions[0]["routing path"] = routing_path
        solutions[0]["end-to-end transmission rate"] = f"{end_to_end_bandwidth} Mbps"
        solutions[0]["end-to-end latency"] = f"{latency} ms"
        solutions[0]["absolute path cost"] = f"{absolute_cost}"
        dijkstra_data['best_path'][str(routes[destination])] = {'end_to_end_bandwidth': end_to_end_bandwidth,
                                                                'path_latency': latency,
                                                                'absolute_cost': absolute_cost
                                                               }
    else:
        best_path = dijkstra_data['best_path'][list(dijkstra_data['best_path'].keys())[0]]
        if absolute_cost > best_path['absolute_cost']:
            solutions[0] = {"source node": f"Node-{routes[destination][0]}"}
            routing_path = ''
            for idx, sensor in enumerate(routes[destination]):
                if idx > 0:
                    routing_path += f"(Node-{sensor}, {get_path_bandwidth_costs(sensor)[routes[destination][idx - 1]]} Mbps),"
            solutions[0]["routing path"] = routing_path
            solutions[0]["end-to-end transmission rate"] = f"{end_to_end_bandwidth} Mbps"
            solutions[0]["end-to-end latency"] = f"{latency} ms"
            solutions[0]["absolute path cost"] = f"{absolute_cost}"
            dijkstra_data['possible_paths'][list(dijkstra_data['best_path'].keys())[0]] = {
                                     'end_to_end_bandwidth': best_path['end_to_end_bandwidth'],
                                     'path_latency': best_path['path_latency'],
                                     'absolute_cost': best_path['absolute_cost']
                                    }
            dijkstra_data['best_path'].pop(list(dijkstra_data['best_path'].keys())[0])
            dijkstra_data['best_path'][str(routes[destination])] = {
                                     'end_to_end_bandwidth': end_to_end_bandwidth,
                                     'path_latency': latency,
                                     'absolute_cost': absolute_cost
                                    }
        else:
            dijkstra_data['possible_paths'][str(routes[destination])] = {'end_to_end_bandwidth': end_to_end_bandwidth,
                                                             'path_latency': latency,
                                                             'absolute_cost': absolute_cost
                                                             }
    # output the best path
    print("Optimal route to "+str(destination)+": "+optimal_route)
    # output its associated best transmission rate
    print("Data Transmission Rate: "+str(end_to_end_bandwidth)+" Mbps")
    # output the total latency
    print("Total Latency: "+str(latency)+" ms")
    # output the path absolute cost
    print("Absolute cost: "+str(absolute_cost))

    return dijkstra_data, solutions

In [516]:
def dijkstra_algorithm(init_node):
    """
    """
    global visited_nodes, unvisited_nodes, routes, known_costs, dijkstra_data, solutions
    dijkstra_data = {'best_path': {},
        'possible_paths': {}}
    solutions = [{}]
    known_costs = {}
    routes = {sensor: [] for sensor in x_y_data.keys()}
    visited_nodes = []
    unvisited_nodes = []
    init_known_costs(init_node)
    current_node = init_node
    i = 1
    # create iteration number variable that will help us know number of sensors visited for purposes of latency computation
    no_visited_nodes = 1
    while unvisited_nodes:
        #print("********************************************************Iter "+str(i)+"***********************************************************")
        next_node, no_visited_nodes = select_max_path(current_node, no_visited_nodes)
        #print("Next best node:"+ str(next_node))
        #print("Unvisited Nodes:"+ str(unvisited_nodes))
        #print("Visited Nodes:"+ str(visited_nodes))
        #print("Known Best Routes:"+ str(routes))
        #print("Known Largest Absolute Costs:"+ str(known_costs))
        current_node = next_node
        i += 1
    dest_nodes = [x_y_base_station_1[0], x_y_base_station_2[0]]
    print("-------------------------------------------------------------------------------------------------------------------------------------")
    for node in dest_nodes:
        dijkstra_data, solutions = get_optimal_route(node, dijkstra_data, solutions)

    return dijkstra_data, solutions